In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('../../results/result.csv')

In [18]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 100000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.float_format')
pd.reset_option('display.max_colwidth')

In [19]:
df.columns

Index(['idx', 'branch_name', 'merge', 'left', 'right', 'base', 'parent test',
       'merge test', 'repo_name', 'gitmerge-ort', 'gitmerge-ort-ignorespace',
       'gitmerge-recursive-patience', 'gitmerge-recursive-minimal',
       'gitmerge-recursive-histogram', 'gitmerge-recursive-myers',
       'gitmerge-resolve', 'git-hires-merge', 'spork', 'intellimerge',
       'Equivalent gitmerge-ort gitmerge-ort-ignorespace',
       'Equivalent gitmerge-ort gitmerge-recursive-patience',
       'Equivalent gitmerge-ort gitmerge-recursive-minimal',
       'Equivalent gitmerge-ort gitmerge-recursive-histogram',
       'Equivalent gitmerge-ort gitmerge-recursive-myers',
       'Equivalent gitmerge-ort gitmerge-resolve',
       'Equivalent gitmerge-ort git-hires-merge',
       'Equivalent gitmerge-ort spork', 'Equivalent gitmerge-ort intellimerge',
       'Equivalent gitmerge-ort-ignorespace gitmerge-recursive-patience',
       'Equivalent gitmerge-ort-ignorespace gitmerge-recursive-minimal',
      

In [5]:
df.iloc[13]

idx                                                                        13
branch_name                                                 refs/heads/master
merge                                fdcc5f6727546f88afa663807134c8a02e3460fd
left                                 03e88b7fa53da1d9ce0910922bf50f8a0beccbf3
right                                14bb50651811901405b82d8f82ef8e055ba04e81
                                                       ...                   
gitmerge-recursive-myers run_time                                    0.494134
gitmerge-resolve run_time                                            4.643133
git-hires-merge run_time                                             0.340799
spork run_time                                                        4.65249
intellimerge run_time                                                3.562799
Name: 13, Length: 74, dtype: object

In [6]:
from merge_tester import MERGE_STATE
df['gitmerge-ort'].apply(lambda x: MERGE_STATE[x])[0]

<MERGE_STATE.Tests_passed: 5>

In [12]:
from validate_repos import clone_repo
repo = clone_repo(df.iloc[13]["repo_name"]) # Return a Git-Python repo object

Adobe-Consulting-Services/acs-aem-commons  : Cloning repo
Adobe-Consulting-Services/acs-aem-commons  : Finished cloning


In [13]:
# For a git to work you need to pass a name for each branch
repo.remote().fetch()
repo.git.checkout(df.iloc[13]["left"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "NAME_OF_LEFT_BRANCH", force=True)
repo.git.checkout(df.iloc[13]["right"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "NAME_OF_RIGHT_BRANCH", force=True)

''

In [ ]:
repo_path = repo.git.rev_parse("--show-toplevel")
repo_path

'/Users/benediktschesch/Git/AST-Merging-Evaluation/src/python/repos/Adobe-Consulting-Services/acs-aem-commons'

In [ ]:
import subprocess
# Returns = 1 process fails
subprocess.call(["../scripts/merge_tools/gitmerge-ort.sh",repo_path,"NAME_OF_LEFT_BRANCH","NAME_OF_RIGHT_BRANCH"])

Path: /Users/benediktschesch/Git/AST-Merging-Evaluation/src/python/repos/Adobe-Consulting-Services/acs-aem-commons
Running: git merge --no-edit -s ort NAME_OF_RIGHT_BRANCH
Auto-merging bundle-twitter/pom.xml
Auto-merging bundle/pom.xml
Auto-merging bundle/src/main/java/com/adobe/acs/commons/workflow/process/impl/SyntheticWrapperWorkflowProcess.java
Auto-merging pom.xml
CONFLICT (content): Merge conflict in pom.xml
Automatic merge failed; fix conflicts and then commit the result.


Switched to branch 'NAME_OF_LEFT_BRANCH'


1

In [ ]:
subprocess.call(["cd",repo_path,"&&","git","merge","--abort"])

0

In [ ]:
# Merge Sucess (Return 0)
subprocess.call(["../scripts/merge_tools/gitmerge-ort-ignorespace.sh",repo_path,"NAME_OF_LEFT_BRANCH","NAME_OF_RIGHT_BRANCH"])
# Be careful the merge has been completed the history of the left branch has now changed and you can't 
# merge again unless you delete the repo and clone it again

Path: /Users/benediktschesch/Git/AST-Merging-Evaluation/src/python/repos/Adobe-Consulting-Services/acs-aem-commons
Running: git merge --no-edit -s ort -Xignore-space-change NAME_OF_RIGHT_BRANCH
Auto-merging bundle-twitter/pom.xml
Auto-merging bundle/pom.xml
Auto-merging bundle/src/main/java/com/adobe/acs/commons/workflow/process/impl/SyntheticWrapperWorkflowProcess.java
Auto-merging pom.xml


Already on 'NAME_OF_LEFT_BRANCH'


Merge made by the 'ort' strategy.
 bundle-twitter/pom.xml                                  | 12 ++++++++++++
 bundle/pom.xml                                          | 12 ++++++++++++
 .../httpcache/engine/impl/HttpCacheEngineImpl.java      |  5 ++++-
 .../impl/ReplicateWithOptionsWorkflowProcess.java       |  3 ++-
 .../process/impl/SyntheticWrapperWorkflowProcess.java   |  3 ++-
 pom.xml                                                 | 17 ++++++++++++++++-
 6 files changed, 48 insertions(+), 4 deletions(-)
/Users/benediktschesch/Git/AST-Merging-Evaluation/src/python


0

In [49]:
#  * Merges for which some tools succeeded and others failed.  This is perhaps more interesting for the current paper than for building a new merge tool, but it does indicate what features to be sure to retain from current tools.
df[(~df["Equivalent gitmerge-ort gitmerge-ort-ignorespace"]) & (df["gitmerge-ort"] == "Merge_failed") & (df["gitmerge-ort-ignorespace"] != "Merge_failed")]

,idx,branch_name,merge,left,right,base,parent test,merge test,repo_name,gitmerge-ort,...,gitmerge-ort run_time,gitmerge-ort-ignorespace run_time,gitmerge-recursive-patience run_time,gitmerge-recursive-minimal run_time,gitmerge-recursive-histogram run_time,gitmerge-recursive-myers run_time,gitmerge-resolve run_time,git-hires-merge run_time,spork run_time,intellimerge run_time
13,13,refs/heads/master,fdcc5f6727546f88afa663807134c8a02e3460fd,03e88b7fa53da1d9ce0910922bf50f8a0beccbf3,14bb50651811901405b82d8f82ef8e055ba04e81,962fc9afb92a08904a1e1d645e7ec8802a9588db,Tests_passed,Tests_passed,Adobe-Consulting-Services/acs-aem-commons,Merge_failed,...,0.288749,0.417700,0.555414,0.675056,0.392821,0.494134,4.643133,0.340799,4.652490,3.562799
573,573,refs/heads/master,7422197487f7fb1c43111c98d9859e9c05052566,668d6eb9dd831ffaa0a5f20dd9f34f6e54eb8a08,73a0bfe83fe56f264293476ce262a774c7b7fc1f,1b1378752ee708235b1cf49e4c93a228e3832841,Tests_passed,Tests_passed,JavaMoney/jsr354-api,Merge_failed,...,0.173899,0.122049,0.147904,0.111410,0.311316,1.023735,1.627475,0.104198,5.933104,1.179602
1049,1049,refs/heads/master,f2900aa41cfb1fef3eafa17f81afe244d6998027,505ae7ec23ff08ff55de75e9d0ae8a72de317b46,5ca1542ac00f7538815c422f2577561fabcec239,8ff795e88053ef49dd89b3bc547b9f36f71d054f,Tests_passed,Tests_passed,SonarCommunity/sonar-l10n-zh,Merge_failed,...,0.078984,0.283356,0.093402,0.084763,0.085285,0.082006,0.714700,0.074157,0.072072,0.912455
1061,1061,refs/heads/master,ef3414c1b836b2336081c8c40dadf8586b90f65f,fadb5bb352efc6f2f7ee50e37111de9e1d03ba12,5ba4c4ede75791b9fb75ac0342b2faf85532bd0f,84a7197023379d7a6e2d686c83f9aa6512b19abe,Tests_passed,Tests_passed,SonarCommunity/sonar-l10n-zh,Merge_failed,...,0.070356,0.066067,0.040771,0.036324,0.075095,0.071372,1.111429,0.052421,0.063754,0.859368
1448,1448,refs/heads/master,762c924cf6a187f513bafe97d4702d4377bab655,81fcbeafcf281ce07fa90b0ac58876c59e569ff1,639362caa32daa89f24622194b78f6a3020982d2,ac828a3554a8737139fba3b2e9328f2b917042f5,Tests_passed,Tests_failed,apache/olingo-odata4,Merge_failed,...,1.137488,0.375888,1.635434,1.192263,0.778543,0.645703,1.547162,0.285200,38.312896,3.823840
1920,1920,refs/heads/old-master-before-asf,c3d344324564a68fdf91b0f2bca3574c3cf80951,e6b93fc8680e0d2cbba50b83c15767e6a4e64916,f3bf1b79225f243e7ebf6ebd0c8d455cc64aca00,7e766dc12dd5bd091da305b120c638a78ec872cb,Tests_passed,Tests_passed,commons-rdf/commons-rdf,Merge_failed,...,0.050469,0.078570,0.045877,0.044493,0.047837,0.047098,33.328239,0.035046,3.359272,0.842363
1984,1984,refs/heads/master,0bce3a1367ac611fc90a265cee3fc6de53c75943,3330e18f01f39361c7cfbb21d26d44c8f9c21a00,2007bf5f639ec25ca7b30b77ffd3f9701923a57e,49bceca9c2a667afb31bff50f72c5e31b80bcd43,Tests_passed,Tests_passed,danielflower/maven-gitlog-plugin,Merge_failed,...,0.283195,0.070959,0.117276,0.070355,0.055536,0.491101,0.666561,0.045627,8.695772,1.053203
2469,2469,refs/heads/master,c93cbfe5d62bad1f0c8477f9510464dab3d38854,ce0aa5fddd745e8374e54e5902955d303fbb4341,7ddafe692c2b876757653759fb6bbdc496b98e73,3a8f4747f322d7373afe1c456a224c5c9e512f00,Tests_passed,Tests_failed,ffpojo/ffpojo,Merge_failed,...,0.075269,0.097271,0.164239,0.118684,0.120120,0.168207,0.406082,0.079607,5.093870,1.887660
3595,3595,refs/remotes/origin/pull/35,bc4a9ceacb2b36ad54f14ff7ca31fe9329158728,f7e737288e1181f48dd3d04577d42a5c46cb7507,57eafb53f1354d114d2af329bbd51d064ca1d269,71cbd4bb4f6bc9b0e15ccfa6c644875ab3aa4153,Tests_passed,Tests_passed,jhy/jsoup,Merge_failed,...,0.064064,0.075176,0.088486,0.081589,0.096748,0.117634,0.388403,0.062407,7.726159,1.258215
3667,3667,refs/heads/master,2a02a3b654e63d30b81a9295155e2121c017f983,5284ad3f0505a5b03fcfa60fad14df03edf657ef,843d103aa8127d67e6863f8c5d5ac89805e293a3,3642654c4afd5013cdc81e1a6ce2d42ddf93f777,Tests_passed,Tests_passed,jline/jline2,Merge_failed,...,0.061602,0.127097,0.061299,0.049168,0.056159,0.061359,1.499069,0.057555,5.744925,0.930597
